<a href="https://colab.research.google.com/github/Mr-RedBeard/Projects/blob/main/Finished_CallerID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**** TO MERGE NEW JSON FILES INTO ONE JSON FILE. THIS HAS TO HAPPEN BEFORE THEY CAN BE CONVERTED INTO A DATAFRAME. ****

In [ ]:
import re
import json
import os


def merge_json_files(original_file, new_files, output_file):
    # Load the original JSON file
    with open(original_file, 'r') as original:
        merged_data = json.load(original)

    # Iterate through each new JSON file
    for new_file in new_files:
        # Load the new JSON file
        with open(new_file, 'r') as file:
            data = json.load(file)
            # Append the 'cbNumbers' list from the new file to the original data
            merged_data['cbNumbers'].extend(data.get('cbNumbers', []))

    # Save the merged data to a new JSON file named 'blocked_numbers.json'
    with open(output_file, 'w') as outfile:
        json.dump(merged_data, outfile, indent=4)

# -----------------------------------------------------------------------------

# *** COPY AND PASTE THE OLD JSON WITH NEW JSON. ORIGINAL_FILE IS THE CURRENT ***
# *** BLOCKED_NUMBER.JSON THE NEW_FILE IS THE NEW JSON YOU WANT TO ADD TO IT. ***



original_file = '/content/1712617286789.acr-backup'



new_files = ['/content/Auto_backup-1712439214126.acr-backup']

output_file = '/content/complete_blocked_numbers.json'

#-------------------------------------------------------------------------------

#output_file = original_file

merge_json_files(original_file, new_files, output_file)





**** CONVERTING JSON INTO A DATAFRAME AND SAVING IT AS A WHOLE TO A CSV FILE FOR EASY USE. *****

In [ ]:
import pandas as pd
import re
import json
import os

#---------------------------------------------------------------------



#***THIS IS WHERE YOU COPY AND PASTE THE JSON FILE PATH***
JSON_FILE_PATH = '/content/complete_blocked_numbers.json'
CSV_FILE_PATH = '/content/complete_blocked_numbers.csv'



#---------------------------------------------------------------------

def load_json(file_path):
    """
    Load JSON data from the specified file path.

    Parameters:
    - file_path (str): Path to the JSON file.

    Returns:
    - dict: JSON data loaded from the file.
    """
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print("JSON file not found.")
        return None
    except json.JSONDecodeError:
        print("Invalid JSON format.")
        return None

def extract_data(json_data):
    """
    Extract 'number' and 'notes' fields from the JSON data.

    Parameters:
    - json_data (dict): JSON data containing the 'cbNumbers' key.

    Returns:
    - list of tuples: Extracted 'number' and 'notes' fields.
    """
    if 'cbNumbers' not in json_data:
        print("No 'cbNumbers' key found in the JSON data.")
        return []

    extracted_data = []
    for entry in json_data['cbNumbers']:
        number = entry.get('number', 'N/A')
        notes = entry.get('notes', 'N/A')
        number = number.replace('+', '')  # Remove '+' sign if present
        extracted_data.append((number, notes))
    return extracted_data

def create_dataframe(extracted_data):
    """
    Create a pandas DataFrame from the extracted data.

    Parameters:
    - extracted_data (list of tuples): Extracted 'number' and 'notes' fields.

    Returns:
    - DataFrame: Pandas DataFrame containing 'number' and 'notes' columns.
    """
    df = pd.DataFrame(extracted_data, columns=['Number', 'Notes'])
    return df

def main():
    # Load JSON data
    json_data = load_json(JSON_FILE_PATH)
    if json_data is None:
        return

    # Extract data
    extracted_data = extract_data(json_data)

    # Create DataFrame
    df = create_dataframe(extracted_data)

    # Convert 'Number' column to integers
    df['Number'] = df['Number'].astype(int)

    # Define a function to remove the first digit if it is '1'
    def remove_first_digit_if_one(number):
      number_str = str(number)
      if number_str[0] == '1':
        return int(number_str[1:])
      return number

    # Apply the function to the 'Number' column
    df['Number'] = df['Number'].apply(remove_first_digit_if_one)

    # Write DataFrame to CSV
    df.to_csv('omplete_blocked_numbers.csv', index=False)
    print("Updated DataFrame written to 'complete_blocked_numbers.csv'.")

if __name__ == "__main__":
    main()

Updated DataFrame written to 'complete_blocked_numbers.csv'.


**** THIS IS WHERE YOU WILL ADD THE .CSV FILE FROM VIOPO AND THE .XLS FILE FROM VOIPMS TO BE SORTED AND CHANGED INTO CSV FOR YOUR USE. ****

In [ ]:
import pandas as pd
import re
from datetime import date

def process_call_log(file_path, blocked_numbers):
    if file_path == file_path_VO:
        vo_data = pd.read_csv(file_path_VO)
        # Extract required columns and filter out rows with blocked Caller-ID
        vo_call_data = vo_data.loc[:, ['Direction', 'Caller-ID', 'Call From', 'Call To', 'Start Time', 'Duration']]
        # Filter out rows with non-integer values in 'Call From' column
        vo_call_data = vo_call_data[pd.to_numeric(vo_call_data['Call From'], errors='coerce').notnull()]
        # Transform 'Call From' column to integer dtype
        vo_call_data['Call From'] = vo_call_data['Call From'].astype(int)
        blocked_numbers_int = blocked_numbers_df.iloc[:,0].astype(int)
        # Filter out rows with 'Call From' matching blocked numbers
        vo_received_call = vo_call_data[~vo_call_data['Call From'].isin(blocked_numbers_int)]
        # Remove rows with 'Outgoing' in the 'Direction' column
        vo_received_call = vo_received_call[vo_received_call['Direction'] != 'Outgoing']
        # Take a total of all duplicate entries in the 'Call From' column and add a 'Number of Calls' column
        vo_received_call['Number of Calls'] = vo_received_call.groupby('Call From')['Call From'].transform('count')
        # Keep only the first entry for each 'Call From'
        vo_received_call = vo_received_call.drop_duplicates(subset=['Call From'])
        # Extract date from 'Start Time' column
        vo_date = date.today().strftime('%m-%d-%Y')
        # Write cleaned data to a new CSV file
        vo_output_file = f"{vo_date}_VO_calls.csv"
        vo_received_call.to_csv(vo_output_file, index=False)
        print(f"Filtered data written to {vo_output_file}")

    elif file_path == file_path_MS:
      # Read MS file
      ms_data = pd.read_csv(file_path_MS)
      # Extract required columns and filter out rows with blocked CallerID
      ms_call_data = ms_data.loc[:,['Date', 'CallerID', 'Destination', 'Description', 'Seconds']]

      # Extract and keep only the first 10 digits from 'CallerID'
      ms_call_data['CallerIDs'] = ms_call_data['CallerID'].str.extract(r'(\d{10})')
      # If no 10 digits found, try to find numbers enclosed within angle brackets
      ms_call_data['CallerIDs'].fillna(ms_call_data['CallerID'].str.extract(r'<(\d{10})>')[0], inplace=True)
      # Remove any non-digit characters
      ms_call_data['CallerIDs'] = ms_call_data['CallerIDs'].str.replace(r'\D', '')

      # Now drop the original 'CallerID' column
      ms_call_data.drop(columns=['CallerID'], inplace=True)


      # Reset index of ms_call_data
      ms_call_data.reset_index(drop=True, inplace=True)

      # Checking call against blocked number list
      blocked_numbers_int = blocked_numbers_df.iloc[:,0].astype(int)
      ms_received_call = ms_call_data[~ms_call_data['CallerIDs'].isin(blocked_numbers_int.tolist())]

      if 'Number' in blocked_numbers_df.columns:
        # Extract and convert blocked CallerID information to a list
        blocked_caller_ids = blocked_numbers_df['Number'].tolist()
        # Filter out rows with blocked CallerID
        ms_received_call = ms_call_data[~ms_call_data['CallerIDs'].isin(blocked_caller_ids)]
        # Filter rows with 'Inbound DID' in the 'Description' column
        ms_received_call = ms_received_call[ms_received_call['Description'] == 'Inbound DID']

        # Count the total number of occurrences for each unique 'CallerID_numeric' before cleaning
        ms_received_call['Number of Calls'] = ms_received_call.groupby('CallerIDs')['CallerIDs'].transform('count')
        # Drop duplicate entries based on the first unique entry
        ms_received_call = ms_received_call.drop_duplicates(subset=['CallerIDs'], keep='first')

        # Checking call against blocked number list
        blocked_numbers_int = blocked_numbers_df['Number'].astype(int)
        ms_received_call = ms_received_call[~ms_received_call['CallerIDs'].isin(blocked_numbers_int.tolist())]
        # Assign the cleaned 10-digit numbers back to the 'CallerID' column
        ms_received_call['CallerID'] = ms_received_call['CallerIDs']

        # Extract date from 'Date' column
        ms_date = date.today().strftime('%m-%d-%Y')
        # Write cleaned data to a new file
        ms_output_file = f"{ms_date}_MS_calls.csv"
        ms_received_call.to_csv(ms_output_file, index=False)
        print(f"Filtered data written to {ms_output_file}")
      else:
        print("Blocked numbers data frame does not contain 'Number' column.")




    else:
        print("Unsupported file format.")


#-------------------------------------------------------------------------------
# HOW TO USE THIS PROGRAM

file_path_VO = "/content/call-logs-2024-04-08.csv"  # Replace with your CSV file path
file_path_MS = "/content/cdrq32p6jf8v63c_2024-04-09_2024-04-09.csv"  # Replace with your CSV file path
blocked_numbers_path = "/content/omplete_blocked_numbers.csv"  # Example blocked numbers

#-------------------------------------------------------------------------------

blocked_numbers_df = pd.read_csv(blocked_numbers_path)  # Load the blocked numbers CSV into a dataframe

# Call process_call_log function with CSV file path
process_call_log(file_path_VO, blocked_numbers_df)

# Call process_call_log function with Excel file path and engine parameter
process_call_log(file_path_MS, blocked_numbers_df)

Filtered data written to 04-10-2024_VO_calls.csv
Filtered data written to 04-10-2024_MS_calls.csv
